In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
llm = ChatOpenAI(model="gpt-4o-mini")

/Users/ryu/dev-study/llm/langchain/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
  separator="\n",
  chunk_size=1, # 줄바꿈 기준으로 나누기 위해서 0으로 처리 하려고 했으나 0 이상이어야함
  chunk_overlap=0
)

loader = TextLoader("./docs/travel.txt", encoding="utf-8")
documents = loader.load_and_split(text_splitter=text_splitter)

Created a chunk of size 52, which is longer than the specified 1
Created a chunk of size 46, which is longer than the specified 1
Created a chunk of size 43, which is longer than the specified 1
Created a chunk of size 47, which is longer than the specified 1
Created a chunk of size 41, which is longer than the specified 1
Created a chunk of size 41, which is longer than the specified 1
Created a chunk of size 47, which is longer than the specified 1
Created a chunk of size 38, which is longer than the specified 1
Created a chunk of size 40, which is longer than the specified 1
Created a chunk of size 41, which is longer than the specified 1
Created a chunk of size 38, which is longer than the specified 1
Created a chunk of size 46, which is longer than the specified 1
Created a chunk of size 36, which is longer than the specified 1
Created a chunk of size 35, which is longer than the specified 1
Created a chunk of size 45, which is longer than the specified 1
Created a chunk of size 3

In [4]:
from langchain_chroma import Chroma
db = Chroma.from_documents(documents=documents, embedding=hf_embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [5]:
retriever = db.as_retriever(
  search_type="mmr",
  search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.5}
)

In [6]:
from langchain_core.prompts import ChatPromptTemplate

message = """
Answer this question using the provided context only.
아래 제공된 내용만 참고하여 사용자의 질문에 답변해줘. 모르면 솔직하게 그냥 모른다고 답해

Context:
{context}

User Input:
{input}
"""

prompt_template = ChatPromptTemplate.from_messages([
  ("human", message)
])

In [ ]:
from langchain import hub
# https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
prompt_template = hub.pull("langchain-ai/retrieval-qa-chat")
prompt_template

/Users/ryu/dev-study/llm/langchain/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [9]:
from langchain_core.runnables import RunnablePassthrough

# question = "여행 계획 세우기"
# context = retriever.invoke(question)
# question = RunnablePassthrough().invoke(question)
# data = {
#   "context" : context,
#   "input": question
# }
data = {
  "context" : retriever,
  "input" : RunnablePassthrough() # 사용자 입력값 들어감
}

rag_chain = data | prompt_template | llm | parser

In [11]:
rag_chain.invoke("이색적인 숙소에는 어떤 곳이 있을까?")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "이색적인 숙소에는 어떤 곳이 있을까?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,input>] Entering Chain run with input:
{
  "input": "이색적인 숙소에는 어떤 곳이 있을까?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,input> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "이색적인 숙소에는 어떤 곳이 있을까?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,input> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "이색적인 숙소에는 어떤 곳이 있을까?"
}


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,input>] [1.05s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer any use questions based solely on the context below:\n\n<context>\n[Document(id='c6dc65cd-1a1d-4df6-92b4-4c365bdf202f', metadata={'source': './docs/travel.txt'}, page_content='이색적인 숙소에서 머물기. 트리하우스, 동굴 호텔, 빙하 호텔 등 독특한 숙소 체험.'), Document(id='6c4683a0-7ca5-4f60-ac32-1c4003ba1921', metadata={'source': './docs/travel.txt'}, page_content='한적한 여행지 추천. 관광객이 몰리지 않는 조용하고 아름다운 여행지를 찾아보자.'), Document(id='fe66cea5-b051-4be2-ae65-c701d1e07ed4', metadata={'source': './docs/travel.txt'}, page_content='배낭여행자를 위한 추천 도시. 예산이 적어도 충분히 즐길 수 있는 세계 곳곳의 명

'이색적인 숙소로는 트리하우스, 동굴 호텔, 빙하 호텔 등이 있습니다. 이러한 독특한 숙소에서 특별한 경험을 할 수 있습니다.'

In [12]:
rag_chain.invoke("우주의 역사에 대해서 알려줘")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "우주의 역사에 대해서 알려줘"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,input>] Entering Chain run with input:
{
  "input": "우주의 역사에 대해서 알려줘"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,input> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "우주의 역사에 대해서 알려줘"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,input> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "우주의 역사에 대해서 알려줘"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,input>] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run 

'죄송하지만, 제공된 콘텐츠에는 우주의 역사에 대한 정보가 포함되어 있지 않습니다. 대신 여행 관련 자료에 대해 질문해 주시면 도와드리겠습니다.'

In [10]:
import langchain
langchain.debug = True